In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_02 import *

In [3]:
path= Path('../PCImages')
path2fm= '../FeatureMatsMerged/TheGreatCollection.txt'
path2colnames= 'FeatureMatIndex.txt'

In [4]:
fm= fm_from_txt(path2fm, path2colnames)
fm.shape

(56657, 105)

In [165]:
cols= ['id','date','time','label1','label2','label3','ind','wallcrash','temp','LP','HL',
       't_stretch','t_relax','framecut','fps','medium','passage', 'ms_ch1','mf_ch1',
       'mf_real_ch1','cv_nn_ch1','cvn_ch1','ms_ch2','mf_ch2','mf_real_ch2','cv_nn_ch2','cvn_ch2']
cols_filtered= [c for c in fm.columns if c not in cols]
cols_filtered= [c for c in cols_filtered if '2s' not in c]
len(cols_filtered)

44

In [188]:
#export
def get_filtered_cols(df, cols2keep=None):
    if cols2keep is None:
        cols= ['id','date','time','label1','label2','label3','ind','wallcrash','temp','LP','HL',
           't_stretch','t_relax','framecut','fps','medium','passage', 'ms_ch1','mf_ch1',
           'mf_real_ch1','cv_nn_ch1','cvn_ch1','ms_ch2','mf_ch2','mf_real_ch2','cv_nn_ch2','cvn_ch2']
    else: cols= cols2keep
    cols= [c for c in df.columns if c not in cols]
    return [c for c in cols if '2s' not in c]

In [186]:
cols_filtered= get_filtered_cols(fm)
len(cols_filtered)

44

In [1]:
#export
class CellDataSet(Dataset):
    def __init__(self, path, fm, cols2keep, label = 'label1', cell_phase = None, trfms=None, 
                 random_sample=None):
        self.fm = fm
        self.cols2keep= cols2keep
        self.y = self.fm[label]
        self.id2label= id2label(self.fm.id, self.y)
        if cell_phase is None:
            self.data_files = get_filtered_files(path, img_ids=self.fm.id, labels=self.y) 
        else: 
            self.data_files= [
                f for f in get_filtered_files(path, img_ids=self.fm.id, labels=self.y) if cell_phase in str(f)
            ]
        if random_sample is not None: self.data_files= random.sample(self.data_files, random_sample)
        self.trfms = get_trfms(trfms)
        
    def __getindex__(self, idx):
        return load_file(self.data_files[idx])

    def __len__(self):
        return len(self.data_files)
    
    def get_row(self, img_id):
        x_np= np.array(self.fm.loc[self.fm['id'].isin([img_id])][self.cols2keep]).flatten() 
        return torch.cuda.FloatTensor(x_np)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name  = self.data_files[idx]
        img_id, phase = split_fn(img_name)
        label=          self.id2label[img_id]
        image =         PIL.Image.open(img_name)
        fm_row=         self.get_row(img_id)

        if self.trfms:
            image = self.trfms(image)

        return image, fm_row, label

NameError: name 'Dataset' is not defined

In [21]:
#export
import torch.nn.functional as F

# Concatenate models:

In [23]:
#export
class Learner():
    def __init__(self, dataset, model):
        self.ds= dataset 
        self.img_size, self.c= self.ds[0][0].shape[0], len(np.unique(self.ds.y))
        self.model= model #get_model(model.cuda(), image_size=self.img_size, c=self.c)
        self.loss= nn.CrossEntropyLoss()
        
    def fit(self, epochs=1, bs=32, lr = 1e-5):
        opt= torch.optim.Adam(self.model.parameters(), lr=lr)
        train_dl, valid_dl= ds2dls(self.ds, bs=bs)
        for epoch in range(epochs):
            self.model.train()
            for img_xb, data_xb, yb in train_dl:
                img_xb=  img_xb.to(device)
                data_xb= data_xb.to(device)
                yb= yb.to(device)
                loss = self.loss(self.model(img_xb, data_xb), yb)
                loss.backward()
                opt.step()
                opt.zero_grad()

            self.model.eval()
            with torch.no_grad():
                tot_loss,tot_acc = 0.,0.
                for img_xb, data_xb, yb in valid_dl:
                    img_xb=  img_xb.to(device)
                    data_xb= data_xb.to(device)
                    yb= yb.to(device)
                    pred = self.model(img_xb, data_xb)
                    pred= pred.to(device)
                    tot_loss += self.loss(pred, yb)
                    tot_acc  += accuracy(pred,yb)
            nv = len(valid_dl)
            print(epoch, tot_loss/nv, tot_acc/nv)
        #return tot_loss/nv, tot_acc/nv

## Model for the feature matrix: 

In [166]:
class FeatureMatrixModel(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.n_in = n_in
        self.lin1 = nn.Linear(self.n_in, 200)
        self.lin2 = nn.Linear(200, 70)
        self.lin3 = nn.Linear(70, n_out)
        self.bn1 = nn.BatchNorm1d(self.n_in)
        self.bn2 = nn.BatchNorm1d(200)
        self.bn3 = nn.BatchNorm1d(70)
        self.drops = nn.Dropout(0.3)
        

    def forward(self, x_cont):
        x = self.bn1(x_cont)
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn3(x)
        x = self.lin3(x)
        return x

In [169]:
class CombinedModel(nn.Module):
    def __init__(self, n_out_cnn=50, n_out_data=50):
        super(CombinedModel, self).__init__()
        self.n_out_cnn, self.n_out_data= n_out_cnn, n_out_data
        self.n_out= self.n_out_cnn + self.n_out_data
        self.n_classes= 7
        
        self.fc0= Resize(350).cuda()
        self.cnn = models.resnet34(pretrained=True).cuda()
        self.cnn.conv1= nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).cuda()
        self.cnn.fc= nn.Linear(in_features=512, out_features=self.n_out_cnn).cuda()
        self.fm_net= FeatureMatrixModel(len(cols_filtered), self.n_out_data).cuda()
        self.fc1 = nn.Linear(self.n_out, self.n_classes).cuda()
        
    def forward(self, image, data):
        x1 = self.cnn(self.fc0(image))
        x2 = self.fm_net(data)
        x = torch.cat((x1, x2), dim=1) 
        x = F.relu(x)
        x = self.fc1(x)
        return x

### Ratio of model outputs:

In [170]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(50,10))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.2839, device='cuda:0') tensor(0.6000, device='cuda:0')


In [171]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(100,10))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.2610, device='cuda:0') tensor(0.5950, device='cuda:0')


In [172]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(100,100))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.3976, device='cuda:0') tensor(0.5750, device='cuda:0')


In [173]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(20,10))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.5031, device='cuda:0') tensor(0.5750, device='cuda:0')


In [174]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(50,40))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.4958, device='cuda:0') tensor(0.5950, device='cuda:0')


In [175]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(50,20))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.4537, device='cuda:0') tensor(0.5900, device='cuda:0')


### Dropout:

In [177]:
#export
class FeatureMatrixModel(nn.Module):
    def __init__(self, n_in, n_out, p):
        super().__init__()
        self.n_in = n_in
        self.lin1 = nn.Linear(self.n_in, 200)
        self.lin2 = nn.Linear(200, 70)
        self.lin3 = nn.Linear(70, n_out)
        self.bn1 = nn.BatchNorm1d(self.n_in)
        self.bn2 = nn.BatchNorm1d(200)
        self.bn3 = nn.BatchNorm1d(70)
        self.drops = nn.Dropout(p)
        

    def forward(self, x_cont):
        x = self.bn1(x_cont)
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn3(x)
        x = self.lin3(x)
        return x

In [191]:
#export
class CombinedModel(nn.Module):
    def __init__(self, n_out_cnn=50, n_out_data=50, p=0.5, n_features=44):
        super(CombinedModel, self).__init__()
        self.n_features= n_features
        self.n_out_cnn, self.n_out_data= n_out_cnn, n_out_data
        self.n_out= self.n_out_cnn + self.n_out_data
        self.n_classes= 7
        
        self.fc0= Resize(350).cuda()
        self.cnn = models.resnet34(pretrained=True).cuda()
        self.cnn.conv1= nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).cuda()
        self.cnn.fc= nn.Linear(in_features=512, out_features=self.n_out_cnn).cuda()
        self.fm_net= FeatureMatrixModel(n_features, self.n_out_data, p).cuda()
        self.fc1 = nn.Linear(self.n_out, self.n_classes).cuda()
        
    def forward(self, image, data):
        x1 = self.cnn(self.fc0(image))
        x2 = self.fm_net(data)
        x = torch.cat((x1, x2), dim=1) 
        x = F.relu(x)
        x = self.fc1(x)
        return x

In [179]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(50,10, p=0.5))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.4163, device='cuda:0') tensor(0.6500, device='cuda:0')


In [180]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(50,10, p=0.7))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.3898, device='cuda:0') tensor(0.6800, device='cuda:0')


In [181]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(50,10, p=0.8))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(1.4486, device='cuda:0') tensor(0.6850, device='cuda:0')


In [182]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(50,10, p=0.8))
learn.fit(8, bs=8, lr=1e-5)

0 tensor(1.4012, device='cuda:0') tensor(0.5650, device='cuda:0')
1 tensor(1.1414, device='cuda:0') tensor(0.5950, device='cuda:0')
2 tensor(1.0009, device='cuda:0') tensor(0.6250, device='cuda:0')
3 tensor(0.9223, device='cuda:0') tensor(0.6650, device='cuda:0')
4 tensor(0.8934, device='cuda:0') tensor(0.6750, device='cuda:0')
5 tensor(0.8145, device='cuda:0') tensor(0.7250, device='cuda:0')
6 tensor(1.0381, device='cuda:0') tensor(0.6350, device='cuda:0')
7 tensor(0.7669, device='cuda:0') tensor(0.7300, device='cuda:0')


In [183]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, CombinedModel(50,10, p=0.5))
learn.fit(8, bs=8, lr=1e-5)

0 tensor(1.3394, device='cuda:0') tensor(0.6050, device='cuda:0')
1 tensor(1.0431, device='cuda:0') tensor(0.6500, device='cuda:0')
2 tensor(0.9430, device='cuda:0') tensor(0.7150, device='cuda:0')
3 tensor(0.8624, device='cuda:0') tensor(0.7050, device='cuda:0')
4 tensor(0.8077, device='cuda:0') tensor(0.7400, device='cuda:0')
5 tensor(0.7854, device='cuda:0') tensor(0.7350, device='cuda:0')
6 tensor(0.7664, device='cuda:0') tensor(0.7450, device='cuda:0')
7 tensor(0.7446, device='cuda:0') tensor(0.7500, device='cuda:0')


In [184]:
ds= CellDataSet(path, fm, cols_filtered, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=None)
learn= Learner(ds, CombinedModel(50,10, p=0.5))
learn.fit(1, bs=8, lr=1e-5)

0 tensor(0.2452, device='cuda:0') tensor(0.9099, device='cuda:0')


In [192]:
!python notebook2script.py 03_combined_model.ipynb

Converted 03_combined_model.ipynb to exp/nb_03.py
